# dynamo query table

In [9]:
import os
import json
import time
import boto3
import tweepy
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session
from dynamodb_json import json_util
from boto3.dynamodb.conditions import Key, Attr
# from tweet_grab

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()

auth = tweepy.OAuthHandler(creds['twitter-api-key'], creds['twitter-secret-key'])
auth.set_access_token(creds['twitter-access-token'], creds['twitter-secret-access'])

api = tweepy.API(auth)

client = boto3.client('dynamodb',
  endpoint_url = "http://localhost:8000",
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            endpoint_url = "http://localhost:8000", \
            region_name='us-east-2')

tweets = dynamodb.Table("tweets")
print(tweets.item_count)
    
print("imported modules.")

216341
imported modules.


## get following

In [15]:
os.getcwd()

'/home/steven/twitter.ai/notebooks'

In [16]:
os.chdir('/home/steven/twitter.ai/main')
import tweet_grab

os.chdir('/home/steven/twitter.ai/notebooks')

imported modules successfully.


In [19]:
try:
    my_following = tweet_grab.get_followers("xanax_princess_", api)

except Exception as e:
    print("Exception: {}".format(e))

got most recent 100 following
got next 100 following
got next 100 following
got next 100 following
got next 100 following


## query for users

In [7]:
df = tweets.query(KeyConditionExpression=Key('user').eq(user))
df["Count"]

0

## scan table

need to paginage results for full table

https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Scan.html

In [19]:
response = tweets.scan()

In [18]:
response.get("LastEvaluatedKey", None)

{'created_at': '2023-04-24T20:50:27Z', 'user': 'javavvitch'}

In [13]:
[k for k in response.keys()]

['Items', 'Count', 'ScannedCount', 'LastEvaluatedKey', 'ResponseMetadata']

In [14]:
response["Count"]

5296

In [16]:
response["LastEvaluatedKey"]

{'created_at': '2023-04-24T20:50:27Z', 'user': 'javavvitch'}

## response items

In [3]:
response["Items"]

[{'favorites': Decimal('2'),
  'created_at': '2017-09-30T01:44:27Z',
  'tweet': 'Two spoons. https://t.co/KXl5ToyRBC',
  'id': Decimal('913942585181396998'),
  'retweets': Decimal('0'),
  'user': 'BEEEZZS',
  'inserted_at': '2023-04-28T18:40:59Z'},
 {'favorites': Decimal('6'),
  'created_at': '2017-10-04T01:28:06Z',
  'tweet': "I should be @ the pitbull concert right now. That's me. That's my zone. Hotel Motel is the gr8st song of all time and I firmly believe that.",
  'id': Decimal('915388023281504256'),
  'retweets': Decimal('0'),
  'user': 'BEEEZZS',
  'inserted_at': '2023-04-28T18:40:59Z'},
 {'favorites': Decimal('2'),
  'created_at': '2017-10-11T02:38:36Z',
  'tweet': 'being 24 i guess is looking for shoes with arch support and still wanting even the ugly ones',
  'id': Decimal('917942478187237377'),
  'retweets': Decimal('0'),
  'user': 'BEEEZZS',
  'inserted_at': '2023-04-28T18:40:59Z'},
 {'favorites': Decimal('1'),
  'created_at': '2017-10-19T22:31:19Z',
  'tweet': "@melmphs W

## pandas

In [4]:
df = pd.DataFrame(json_util.loads(response["Items"]))
df.head()

,favorites,created_at,tweet,id,retweets,user,inserted_at
0,2,2017-09-30T01:44:27Z,Two spoons. https://t.co/KXl5ToyRBC,913942585181396998,0,BEEEZZS,2023-04-28T18:40:59Z
1,6,2017-10-04T01:28:06Z,I should be @ the pitbull concert right now. T...,915388023281504256,0,BEEEZZS,2023-04-28T18:40:59Z
2,2,2017-10-11T02:38:36Z,being 24 i guess is looking for shoes with arc...,917942478187237377,0,BEEEZZS,2023-04-28T18:40:59Z
3,1,2017-10-19T22:31:19Z,@melmphs Wendy's!!! Chicken mozzarella yummm,921141740966498304,0,BEEEZZS,2023-04-28T18:40:59Z
4,3,2017-10-21T02:03:50Z,It's kind of crazy how we all know how to play...,921557611249577984,0,BEEEZZS,2023-04-28T18:40:59Z


In [8]:
df.shape

(5296, 7)

In [6]:
len(np.unique(df["user"]))

10

## summary stats

In [5]:
df.groupby(['user']).agg({'created_at': ['min', 'max'], 'tweet': ['count']})

created_at                       tweet
                                  min                   max count
user                                                             
BEEEZZS          2017-09-30T01:44:27Z  2023-02-24T00:38:09Z   595
CaucasianJames   2022-08-13T16:54:09Z  2023-04-28T05:14:16Z   597
NECROMANClNG     2023-03-19T17:21:45Z  2023-04-28T14:23:49Z   591
RedDerby         2017-10-09T15:07:25Z  2019-08-26T15:42:47Z   601
ViviannRutan     2015-10-22T05:23:36Z  2016-08-08T12:05:41Z   435
akdjsglhaksWOO   2016-06-11T20:29:40Z  2017-06-21T14:27:29Z   571
gmailgaze        2023-04-05T12:52:07Z  2023-04-28T19:50:08Z   530
javavvitch       2023-04-10T16:17:14Z  2023-04-24T20:50:27Z   411
knifeplay_music  2017-08-05T00:37:23Z  2023-04-26T21:30:43Z   366
sopharela        2021-07-15T20:05:08Z  2023-04-27T21:09:29Z   599

## unique users

In [7]:
dynamo_users = list(np.unique(df['user']))
dynamo_users

['BEEEZZS',
 'CaucasianJames',
 'NECROMANClNG',
 'RedDerby',
 'ViviannRutan',
 'akdjsglhaksWOO',
 'gmailgaze',
 'javavvitch',
 'knifeplay_music',
 'sopharela']

## find missing users in dynamo

In [20]:
my_following

['bastard_band',
 'datacoves',
 'punished_cait',
 'pdrmnvd',
 'ChaunceyCC',
 'MerrittBaer',
 'jasminericegirl',
 'artisinalsprite',
 'Sahelanth',
 'emilianbold',
 'loige',
 'halsnew',
 'pinkninjajess',
 'Amy_Henderson__',
 'breathingatlast',
 'ladaw89',
 'dankbees7',
 'allisonavani1',
 'YeetToOrbit',
 'CalMeNoble6',
 'somethin_2b',
 'notrealneverwas',
 'pilemusic',
 'krilliamhmacy',
 'jpegmafia',
 'kittywavvy',
 'elena_lape',
 'esevatojotx',
 'rrossjd',
 'kittyxcate',
 'hahnscratch',
 'MischievousMal',
 'casiebones',
 'DigitalBlkHippy',
 'ennntropy',
 'shanley',
 'AnnieisUndead',
 'mollysoda',
 'Aliafonzy43',
 'Carnage4Life',
 'fixedpointfae',
 'bernhardsson',
 'freshhel',
 'kvlly',
 'seanjtaylor',
 'ecto_fun',
 'colborkmundus',
 'dataandme',
 'robinson_es',
 'dniellechelosky',
 'LittleMissData',
 'beeonaposy',
 'JoannaNewSum41',
 'titsay',
 'ZeroSuitCamus',
 'keribla',
 'vboykis',
 'Pdaddynthehyena',
 'coolgirl0nline',
 'endingwithali',
 'JamieMuma',
 'jsedlacekjr',
 'tagabowPHL',
 'y

In [26]:
subset = [a not in dynamo_users for a in my_following]
missing_users = [x for x, y in zip(my_following, subset) if y]
missing_users

['punished_cait',
 'pdrmnvd',
 'MerrittBaer',
 'jasminericegirl',
 'artisinalsprite',
 'Sahelanth',
 'emilianbold',
 'halsnew',
 'pinkninjajess',
 'Amy_Henderson__',
 'breathingatlast',
 'allisonavani1',
 'YeetToOrbit',
 'CalMeNoble6',
 'notrealneverwas',
 'jpegmafia',
 'elena_lape',
 'esevatojotx',
 'rrossjd',
 'kittyxcate',
 'hahnscratch',
 'casiebones',
 'DigitalBlkHippy',
 'ennntropy',
 'shanley',
 'AnnieisUndead',
 'mollysoda',
 'Aliafonzy43',
 'Carnage4Life',
 'fixedpointfae',
 'bernhardsson',
 'freshhel',
 'kvlly',
 'seanjtaylor',
 'ecto_fun',
 'colborkmundus',
 'dataandme',
 'robinson_es',
 'dniellechelosky',
 'LittleMissData',
 'beeonaposy',
 'JoannaNewSum41',
 'titsay',
 'ZeroSuitCamus',
 'keribla',
 'vboykis',
 'Pdaddynthehyena',
 'coolgirl0nline',
 'endingwithali',
 'JamieMuma',
 'jsedlacekjr',
 'tagabowPHL',
 'yrinfinitycrush',
 'osoosoband',
 'sama',
 'youngtiddy',
 'FranziaMom',
 'pilatesdev',
 'mike_julian',
 'knifeplay_music',
 'joneaux',
 'GergelyOrosz',
 'p8stie',
 '